In [1]:
import efficientnet.keras as efn
import tensorflow as tf

from keras.applications import VGG16
from keras import layers, Model, Sequential
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

import numpy as np

Using TensorFlow backend.


In [2]:
!nvidia-smi # about GPU

Fri Jun 26 11:01:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   33C    P8     6W / 180W |      1MiB / 16278MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from tensorflow.python.client import device_lib

In [4]:
device_lib.list_local_devices() # available devices

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13240527254750569738, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7213883600945513428
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6470495000000291307
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16109194445
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17319454152997303573
 physical_device_desc: "device: 0, name: Quadro P5000, pci bus id: 0000:00:05.0, compute capability: 6.1"]

In [5]:
INPUT_SHAPE = (150, 150, 3) # columns, rows, depth

In [6]:
NUM_CLASSES = 2 # multi-class classification

In [7]:
efn_base = VGG16(
    weights='imagenet',
    include_top=False,
    classes=NUM_CLASSES,
    input_shape=INPUT_SHAPE
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [8]:
# efn_base.summary()

In [9]:
# Brand new FC head
model = Sequential()

model.add(efn_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

In [10]:
# model.summary()

In [11]:
efn_base.trainable = False

In [12]:
BS = 32
ENCODING_MODE = 'hot'

In [13]:
from h5imagegenerator import HDF5ImageGenerator

In [14]:
from albumentations import (
    Compose, ShiftScaleRotate, HorizontalFlip
)

In [15]:
# Data augmentation
aug = Compose([
    ShiftScaleRotate(
        shift_limit=0.0225,
        scale_limit=0.06,
        rotate_limit=30
    ),
    HorizontalFlip()
])

In [16]:
# Generators
train_gen = HDF5ImageGenerator(
    src= './v3/train.h5',
    scaler=False,
    labels_encoding=ENCODING_MODE,
    num_classes=NUM_CLASSES,
    augmenter=aug,
    batch_size=BS)

val_gen = HDF5ImageGenerator(
    src= './v3/val.h5',
    scaler=False,
    labels_encoding=ENCODING_MODE,
    num_classes=NUM_CLASSES,
    batch_size=BS)

In [17]:
lr = 1e-3

In [18]:
rlrop = ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', patience = 5, factor = 0.5, min_lr = 1e-6, verbose = 1)

In [19]:
rms = RMSprop(learning_rate=lr)

model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer=rms
)

h = model.fit(
    train_gen,
    validation_data=val_gen,
    callbacks=[rlrop],
    verbose=1,
    # use_multiprocessing=True,
    # workers=6,
    epochs=50
)


Epoch 1/50
2500/2500 [==============================] - 500s 200ms/step - loss: 0.6987 - accuracy: 0.5868 - val_loss: 0.6006 - val_accuracy: 0.6094
Epoch 2/50
2500/2500 [==============================] - 483s 193ms/step - loss: 0.6562 - accuracy: 0.6119 - val_loss: 0.6186 - val_accuracy: 0.6199
Epoch 3/50
2500/2500 [==============================] - 513s 205ms/step - loss: 0.6519 - accuracy: 0.6206 - val_loss: 0.7005 - val_accuracy: 0.6255
Epoch 4/50
2500/2500 [==============================] - 518s 207ms/step - loss: 0.6481 - accuracy: 0.6239 - val_loss: 0.5907 - val_accuracy: 0.6226
Epoch 5/50
   2/2500 [..............................] - ETA: 6:10 - loss: 0.5371 - accuracy: 0.7500

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.117780). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


2500/2500 [==============================] - 525s 210ms/step - loss: 0.6450 - accuracy: 0.6297 - val_loss: 0.5881 - val_accuracy: 0.6275
Epoch 6/50
2500/2500 [==============================] - 528s 211ms/step - loss: 0.6411 - accuracy: 0.6350 - val_loss: 0.6581 - val_accuracy: 0.6279
Epoch 7/50
2500/2500 [==============================] - 538s 215ms/step - loss: 0.6365 - accuracy: 0.6408 - val_loss: 0.6278 - val_accuracy: 0.6306
Epoch 8/50
2500/2500 [==============================] - 542s 217ms/step - loss: 0.6339 - accuracy: 0.6460 - val_loss: 0.6130 - val_accuracy: 0.6280
Epoch 9/50
2500/2500 [==============================] - 535s 214ms/step - loss: 0.6286 - accuracy: 0.6510 - val_loss: 0.6632 - val_accuracy: 0.6279
Epoch 10/50
2500/2500 [==============================] - 540s 216ms/step - loss: 0.6271 - accuracy: 0.6520 - val_loss: 0.6012 - val_accuracy: 0.6299

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 11/50
2500/2500 [==================

KeyboardInterrupt: 